In [1]:
!pip install transformers datasets evaluate rouge_score pandas scikit-learn

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=d76439beb89f2e796d30d80910008ff4d38f57dc7782638113fb17b16da88e1c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency 

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForQuestionAnswering, pipeline
from rouge_score import rouge_scorer
import nltk
from nltk.translate.meteor_score import meteor_score

# Ensure NLTK resources are downloaded
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:

# Load CSV files
train_df = pd.read_csv('https://raw.githubusercontent.com/viswa3024/aiml-capstone-project-email/main/question-answering/Dataset-2/train.csv')
dev_df = pd.read_csv('https://raw.githubusercontent.com/viswa3024/aiml-capstone-project-email/main/question-answering/Dataset-2/dev.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/viswa3024/aiml-capstone-project-email/main/question-answering/Dataset-2/test.csv')

# Convert DataFrames to Datasets
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)
test_dataset = Dataset.from_pandas(test_df)

In [4]:
print(train_dataset[0])
print(dev_dataset[0])
print(test_dataset[0])

{'question': 'What is the role of Siamese networks in domain adaptation?', 'answer': 'Siamese networks can be used to align the representations of source and target domains by minimizing the discrepancy between them, making the model more robust to domain shifts.'}
{'question': 'How do U-Nets skip connections contribute to better segmentation performance?', 'answer': 'Skip connections in U-Net preserve fine-grained spatial information and improving localization and detail in segmentation tasks.'}
{'question': 'What is feature selection?', 'answer': 'The process of selecting a subset of relevant features to improve model performance and reduce complexity.'}


In [5]:
import pandas as pd
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

In [6]:
# Load tokenizer and model
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:01<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
# Tokenize the dataset
def preprocess_function(examples):
    inputs = examples["question"]
    targets = examples["answer"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
dev_dataset = dev_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1985 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

Map:   0%|          | 0/249 [00:00<?, ? examples/s]

In [9]:
import evaluate

In [10]:
rouge_metric = evaluate.load("rouge")

In [11]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)

In [19]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  # Set to "tensorboard" if using TensorBoard
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    #compute_metrics=compute_metrics
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.074200,0.805333
2,0.143800,0.788595
3,0.224200,0.740759
4,0.252500,0.719032
5,0.222600,0.713796
6,0.228900,0.716231
7,0.302200,0.670024
8,0.312900,0.655864
9,0.342700,0.642829
10,0.298300,0.640631


Epoch,Training Loss,Validation Loss
1,0.074200,0.805333
2,0.143800,0.788595
3,0.224200,0.740759
4,0.252500,0.719032
5,0.222600,0.713796
6,0.228900,0.716231
7,0.302200,0.670024
8,0.312900,0.655864
9,0.342700,0.642829
10,0.298300,0.640631


TrainOutput(global_step=3735, training_loss=0.2561572529944071, metrics={'train_runtime': 2615.2439, 'train_samples_per_second': 11.385, 'train_steps_per_second': 1.428, 'total_flos': 4532930297856000.0, 'train_loss': 0.2561572529944071, 'epoch': 15.0})

In [20]:
import numpy as np

In [21]:
import json
import pandas as pd

def save_predictions_and_labels(test_dataset, trainer, file_prefix=''):
    predictions = []
    labels = []

    for batch in trainer.get_test_dataloader(test_dataset):
        outputs = trainer.model(**batch)
        logits = outputs.logits
        pred_ids = np.argmax(logits.detach().cpu().numpy(), axis=-1)

        batch_preds = pred_ids.tolist()
        batch_labels = batch['labels'].tolist()

        predictions.extend(batch_preds)
        labels.extend(batch_labels)

        # Save periodically to manage memory
        if len(predictions) % 1000 == 0:
            pd.DataFrame({'predictions': [json.dumps(p) for p in predictions], 'labels': [json.dumps(l) for l in labels]}).to_csv(f'{file_prefix}_predictions_labels.csv', index=False)

    # Final save
    pd.DataFrame({'predictions': [json.dumps(p) for p in predictions], 'labels': [json.dumps(l) for l in labels]}).to_csv(f'{file_prefix}_predictions_labels.csv', index=False)

save_predictions_and_labels(test_dataset, trainer, 'test')


In [23]:
import pandas as pd
import json
import evaluate
from transformers import T5Tokenizer



# Load ROUGE metric
rouge_metric = evaluate.load("rouge")

def compute_rouge_from_file(file_path):
    df = pd.read_csv(file_path)
    # Convert JSON strings back to lists
    decoded_preds = [json.loads(p) for p in df['predictions']]
    decoded_labels = [json.loads(l) for l in df['labels']]

    # Decode text
    decoded_preds_text = tokenizer.batch_decode(decoded_preds, skip_special_tokens=True)
    decoded_labels_text = tokenizer.batch_decode(decoded_labels, skip_special_tokens=True)

    return rouge_metric.compute(predictions=decoded_preds_text, references=decoded_labels_text)

# Compute ROUGE scores
rouge_scores = compute_rouge_from_file('test_predictions_labels.csv')

print("ROUGE Scores:")
print(rouge_scores)


ROUGE Scores:
{'rouge1': 0.5179784483298742, 'rouge2': 0.2653250931987703, 'rougeL': 0.48884152844948725, 'rougeLsum': 0.4893048657928886}


In [24]:
# Generate predictions
def generate_predictions(test_dataset, trainer):
    predictions = []
    for batch in trainer.get_test_dataloader(test_dataset):
        outputs = trainer.model.generate(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions.extend(decoded_preds)
    return predictions

# Get final output
final_output = generate_predictions(test_dataset, trainer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [25]:
test_df["Generated"] = final_output
test_df.to_csv('t5_base_qna_dataset_2_v1.csv')

In [26]:
df_generated = pd.read_csv('t5_base_qna_dataset_2_v1.csv')

In [27]:
df_generated.head(10)

,Unnamed: 0,question,answer,Generated
0,0,What is feature selection?,The process of selecting a subset of relevant ...,Feature selection is the process of selecting ...
1,1,"What are Markov decision processes (MDP), and ...",MDPs are mathematical frameworks for modeling ...,MDPs are a type of reinforcement learning wher...
2,2,What is batch normalization and how does it work?,Normalizes the layer's input over a mini-batch,Batch normalization normalizes inputs to a hig...
3,3,How does Word2Vec create word embeddings?,Word2Vec creates word embeddings by training o...,Word2Vec uses vectors to represent words in a ...
4,4,What is the role of hidden layers in MLP?,Hidden layers in MLP help capture complex patt...,Hidden layers in MLP allow the model to captur...
5,5,What is a Decision Tree?,A Decision Tree is a predictive learning algor...,A Decision Tree is a supervised learning algor...
6,6,What is variance in machine learning?,Variance refers to the error introduced by the...,Variance measures the difference between predi...
7,7,What are common methods to diagnose bias and v...,Common methods include analyzing learning curv...,Common methods to diagnose bias and variance i...
8,8,What is the difference between L1 and L2 regul...,L1 regularization adds the absolute values of ...,L1 regularization regularizes the inputs of th...
9,9,What is feature engineering for unstructured d...,Feature engineering for unstructured data invo...,Feature engineering for unstructured data invo...


In [28]:
model.save_pretrained("t5_base_qna_dataset_2_v1")
tokenizer.save_pretrained("t5_base_qna_dataset_2_v1")

('t5_base_qna_dataset_2_v1/tokenizer_config.json',
 't5_base_qna_dataset_2_v1/special_tokens_map.json',
 't5_base_qna_dataset_2_v1/spiece.model',
 't5_base_qna_dataset_2_v1/added_tokens.json')

In [29]:
from huggingface_hub import notebook_login

In [30]:
notebook_login()

In [31]:
repo_name="t5_base_qna_dataset_2_v1"

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kkasiviswanath/t5_base_qna_dataset_2_v1/commit/c1853c2deb1184599c00a1a905cbfa4824c3ca3e', commit_message='Upload tokenizer', commit_description='', oid='c1853c2deb1184599c00a1a905cbfa4824c3ca3e', pr_url=None, pr_revision=None, pr_num=None)

In [32]:
from transformers import pipeline

In [33]:

model_name = 'kkasiviswanath/t5_base_qna_dataset_2_v1'

# Load the model and tokenizer using the pipeline
qa_pipeline = pipeline('text2text-generation', model=model_name, tokenizer=model_name)

# Define a question
question = "Can we utilize an autoencoder to perform dimensionality reduction on numerical datasets?"

# Generate a prediction
result = qa_pipeline(question)

# Print the result
print(result)

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'Yes, we can utilize an autoencoder to reduce the dimensionality of numerical dataset'}]


In [34]:
# Define a question
question = "How does Word2Vec create word embeddings?"

# Generate a prediction
result = qa_pipeline(question)

# Print the result
print(result)

[{'generated_text': 'Word2Vec uses vectors to represent words in a vector space, capturing'}]


In [36]:
from transformers import pipeline

# Define the model path
model_name = 'kkasiviswanath/t5_base_qna_dataset_2_v1'

# Load the QA pipeline with your model
qa_pipeline = pipeline('text2text-generation', model=model_name, tokenizer=model_name)

# Define your question
question = "How does Word2Vec create word embeddings"

# Generate the answer with adjusted parameters
result = qa_pipeline(question, max_length=100, num_beams=5, temperature=1.0, top_p=0.9)

# Output the result
print(f"Question: {question}")
print(f"Answer: {result[0]['generated_text']}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Question: How does Word2Vec create word embeddings
Answer: Word2Vec uses vectors to represent words in a vector space, capturing semantic relationships between words and their context.


In [38]:
# Define your question
question = "Can we utilize an autoencoder to perform dimensionality reduction on numerical datasets?"

# Generate the answer with adjusted parameters
result = qa_pipeline(question, max_length=100, num_beams=5, temperature=1.0, top_p=0.9)

# Output the result
print(f"Question: {question}")
print(f"Answer: {result[0]['generated_text']}")

Question: Can we utilize an autoencoder to perform dimensionality reduction on numerical datasets?
Answer: Yes, we can utilize an autoencoder to reduce the dimensionality of numerical datasets by compressing the input data into a lower-dimensional representation.


In [39]:
# Define your question
question = "What is the role of hidden layers in MLP?"

# Generate the answer with adjusted parameters
result = qa_pipeline(question, max_length=100, num_beams=5, temperature=1.0, top_p=0.9)

# Output the result
print(f"Question: {question}")
print(f"Answer: {result[0]['generated_text']}")

Question: What is the role of hidden layers in MLP?
Answer: Hidden layers in MLP allow the model to capture hidden relationships and relationships in data, enabling it to capture hidden patterns and relationships.
